# 2.1.2 Zeichenerklärung – OCR (GPT4o)

* Zero-Shot
* Chain-Of-Thought

In [1]:
from utils.parser import Parser
from utils.openai import OpenAI
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
ava = OpenAI(instructions)
parser = Parser()

In [2]:
# Zeichenerklärung – Schema Nutzungsschablone WITHOUT BOUNDING BOX
# SKIP: ze_raw_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-raw.png"
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE.png"
ze_crop_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-crop.png"

async def run(ze_path):
    base64_ze, img_size = parser.imageToBase64(ze_path)
    print("image_size:", img_size)
    message = ava.request([
        {
            "type": "text",
            "text": 'Gehe Schritt für Schritt vor: 1. Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung. 2. Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone". 3. Gebe die Tabellenerklärung in ASCII-Tabellenform aus.'
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_ze}",
                "detail": "high"
            }
        },
    ])
    return message

messages = await asyncio.gather(run(ze_path), run(ze_crop_path))
for message in messages:
    print("#############################################")
    print(message)

image_size: [768, 1362]
image_size: [768, 1949]
#############################################
1. **Finde den Eintrag "Nutzungsschablone" in der vorliegenden Zeichenerklärung:**

   Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter der Kategorie "Art der Nutzung".

2. **Finde die zugehörige Tabellenerklärung zum Eintrag "Nutzungsschablone":**

   Die Tabellenerklärung zum Eintrag "Nutzungsschablone" lautet:
   ```
   Nutzungsschablone
   § 9 (1) 1 BauGB
   ```

3. **Gebe die Tabellenerklärung in ASCII-Tabellenform aus:**

   Hier ist die Tabellenerklärung in ASCII-Tabellenform:

   ```
   +-------------------+----------------+
   | Nutzungsschablone | § 9 (1) 1 BauGB |
   +-------------------+----------------+
   ```
#############################################
1. Der Eintrag "Nutzungsschablone" befindet sich in der Zeichenerklärung unter den "Planungsrechtliche Festsetzungen". Er ist mit dem Symbol eines Rechtecks mit einer gestrichelten Linie und einem Punkt

In [3]:
# Zeichenerklärung – Schema Nutzungsschablone WITH BOUNDING BOX
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-bb-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

msg4 = ava.request([
    {
        "type": "text",
        "text": 'Gebe die rot umrandete Tabellenerklärung zum Eintrag Nutzungsschablone in ASCII-Tabellenform aus.'
    },
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
])

print(msg4)

image_size: [763, 2000]
Hier ist die rot umrandete Tabellenerklärung in ASCII-Tabellenform:

```
+---------------------+------------------+
| Art der Nutzung     | Gebäudehöhe      |
+---------------------+------------------+
| Grundflächenzahl    | Geschossflächen- |
|                     | zahl             |
+---------------------+------------------+
| Bauweise            |                  |
+---------------------+------------------+
```


In [4]:
# Zeichenerklärung – Komplette Definition Nutzungsschablone
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

ava.addContext([
    {
        "type": "text",
        "text": msg4
    }
])

msg5 = ava.request([
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
        {
        "type": "text",
        "text": 'Fasse alle relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone aus der Zeichenerklärung zusammen. Ausgabe im JSON-Format: {<Parameter>: {"Definition": "", "Zeichen"=""}}'
    }
])

print(msg5)
ava.clearContext()

image_size: [768, 1949]
Hier sind die relevanten Definitionen und Zeichen bezüglich der Parameter der ASCII-Nutzungsschablone aus der Zeichenerklärung im JSON-Format:

```json
{
  "Art der Nutzung": {
    "Definition": "Gewerbegebiete (§ 8 BauNVO)",
    "Zeichen": "GE"
  },
  "Art der Nutzung (eingeschränkt)": {
    "Definition": "Gewerbegebiete eingeschränkt (§ 8 BauNVO)",
    "Zeichen": "GEE"
  },
  "Gebäudehöhe": {
    "Definition": "max. zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe",
    "Zeichen": "GH=max.12,00"
  },
  "Grundflächenzahl": {
    "Definition": "Grundflächenzahl (GRZ) (§ 16 BauNVO)",
    "Zeichen": "0,6"
  },
  "Geschossflächenzahl": {
    "Definition": "Geschossflächenzahl (GFZ) (§ 16 BauNVO)",
    "Zeichen": "1,2"
  },
  "Bauweise": {
    "Definition": "Abweichende Bauweise (§ 22 BauNVO)",
    "Zeichen": "a"
  },
  "Nutzungsschablone": {
    "Definition": "Nutzungsschablone (§ 9 (1) 1 BauGB)",
    "Zeichen": "-"
  }
}
```


In [7]:
# Zeichenerklärung – Inhalt zu den Hauptkategorien
ze_path = "../data/processed/bpläne/2_zeichnung_textteil_getrennt/F11-01-ZE-crop.png"
base64_ze, img_size = parser.imageToBase64(ze_path)
print("image_size:", img_size)

msg6 = ava.request([
    {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{base64_ze}",
            "detail": "high"
        }
    },
    {
        "type": "text",
        "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
    },
])

print(msg6)

image_size: [768, 1949]
Hier sind die extrahierten Informationen zu den angeforderten Themen aus der Planzeichnung:

### Art der baulichen Nutzung
- **Gewerbegebiete (GE)** (§ 8 BauNVO)
- **Gewerbegebiete eingeschränkt (GEE)** (§ 8 BauNVO)

### Maß der baulichen Nutzung
- **Gebäudehöhe (GH)**: max. 12,00 m
- **Grundflächenzahl (GRZ)** (§ 16 BauNVO): 0,6
- **Geschossflächenzahl (GFZ)** (§ 16 BauNVO): 1,2

### Bauweise
- **Abweichende Bauweise** (§ 22 BauNVO)

### Überbaubare Grundstücksfläche
- **Baugrenze** (§ 23 (3) BauNVO)
- **Umgrenzung der Flächen, die von der Bebauung freizuhalten sind** (§ 9 (1) 10 BauGB)

Diese Informationen geben einen Überblick über die planungsrechtlichen Festsetzungen bezüglich der baulichen Nutzung, des Maßes der baulichen Nutzung, der Bauweise und der überbaubaren Grundstücksfläche.


In [ ]:
# Zeichenerklärung – Inhalt zu den Hauptkategorien – WITH CONTEXT
# Kontext: Schema, BauNVO